In [411]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder,RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.metrics import  mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings('ignore')
import matplotlib.pyplot as lib
import catboost
from sklearn.svm import SVR

## Load the dataset
* Dataset from zindi competition

In [359]:
train = pd.read_csv("Free AI Class Hackaton/data/Train.csv")
test = pd.read_csv("Free AI Class Hackaton/data/Test.csv")

In [360]:
dfn = train.copy()
dtn = test.copy()

In [361]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               7205 non-null   object 
 1   Location                7205 non-null   object 
 2   Maker                   7205 non-null   object 
 3   Model                   7205 non-null   object 
 4   Year                    7184 non-null   object 
 5   Colour                  7205 non-null   object 
 6   Amount (Million Naira)  7188 non-null   float64
 7   Type                    7008 non-null   object 
 8   Distance                4845 non-null   object 
dtypes: float64(1), object(8)
memory usage: 506.7+ KB


### Handling Missing Values
There are missing values in the following columns:
* Year
* Amount (Million Naira)
* Type  
* Distance 

from the cell above, the Distance column has almost half of its values missing missing, so the distance column should be removed.
AS for the other columns Type and Year, use a backward-fill to impute missing values.
For the Amount (Million Naira) which is the target column, we can't just fill with any value, we will look for a way to get the missing values

In [362]:
#  backward fill
dfn["Year"] = dfn["Year"].fillna(method="bfill")
dfn["Type"] = dfn["Type"].fillna(method="bfill")

In [363]:
# drop the distance column
dfn.drop(columns="Distance", inplace=True)

In [364]:
# make a copy of the dfn dataframe
dfn_test = dfn.copy()

In [365]:
# removing the null values from the dfn_test and putting them in a variable test_data
test_data = dfn_test[dfn_test["Amount (Million Naira)"].isnull()]

In [366]:
# drop all null values in the dfn_test dataframe
dfn_test.dropna(inplace=True)

In [367]:
test_data.shape, dfn_test.shape

((17, 8), (7188, 8))

#### Getting the Null values in target data("Amount (Million Naira)")
To achieve this, 
1. Make the dfn_test as training set(i.e split into X_train and y_train)
2. Make the test_data the test set since we want to predict the null values 
   * split into X_test, and y_test

In [368]:
# drop the target column("Amount (Million Naira)", and the VehicleID)
X_train = dfn_test.drop(["Amount (Million Naira)", "VehicleID"], axis=1)

In [369]:
# y_train is the target column in the dfn_test data
y_train = dfn_test["Amount (Million Naira)"]

In [370]:
# drop the target column("Amount (Million Naira)", and the VehicleID) in the test data
X_test = test_data.drop(["Amount (Million Naira)", "VehicleID"], axis=1)

In [371]:
# y_train is the target column in the test_data
y_test = test_data["Amount (Million Naira)"]

In [372]:
X_train.shape, y_train.shape

((7188, 6), (7188,))

In [373]:
X_test.shape, y_test.shape

((17, 6), (17,))

In [374]:
# encode the features in X_train and X_testst(turning categories to numerical values)
ohe = OneHotEncoder(handle_unknown="ignore")
X_train = ohe.fit_transform(X_train)
X_test = ohe.transform(X_test)

In [375]:
# use the svm model from sklearn 
svr = SVR()
svr.fit(X_train, y_train)

SVR()

In [376]:
# predict the X_test and pass it in a nariable nan_pred
nan_pred = svr.predict(X_test)

In [377]:
nan_pred

array([ 4.40021863,  4.6970309 ,  6.95777145,  5.37913197,  2.6404342 ,
        6.85611827,  7.10795238,  4.90670271,  9.19555142,  8.899922  ,
        4.3002125 , 12.00902049,  5.69675028,  2.96198268, 20.13949388,
        2.87854262, 16.79889624])

In [378]:
nan_pred.shape

(17,)

In [379]:
# create a new column in the test_data called "nan_pred" and pass it the predicted values(nan_pred)
test_data["nan_pred"] = nan_pred

In [380]:
test_data.shape

(17, 9)

In [381]:
# make the test_data["Amount (Million Naira)"] equal to the nan_pred column and drop the nan_pred column.
test_data["Amount (Million Naira)"]=test_data["nan_pred"]
test_data.drop(["nan_pred"], axis=1, inplace=True)

In [382]:
# concatenate the splitted datasets(i.e test_data and dfn_test) to get back the complete data set
dfn = pd.concat([test_data, dfn_test], axis=0)

In [383]:
# clean train data set with no null values
dfn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7205 entries, 288 to 7204
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               7205 non-null   object 
 1   Location                7205 non-null   object 
 2   Maker                   7205 non-null   object 
 3   Model                   7205 non-null   object 
 4   Year                    7205 non-null   object 
 5   Colour                  7205 non-null   object 
 6   Amount (Million Naira)  7205 non-null   float64
 7   Type                    7205 non-null   object 
dtypes: float64(1), object(7)
memory usage: 506.6+ KB


In [384]:
# Also drop the vehicleid column 
dfn.drop(columns="VehicleID", inplace=True)

In [385]:
dtn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   VehicleID  2061 non-null   object 
 1   Location   2061 non-null   object 
 2   Maker      2061 non-null   object 
 3   Model      2061 non-null   object 
 4   Year       2059 non-null   object 
 5   Colour     2061 non-null   object 
 6   Type       2007 non-null   object 
 7   Distance   1385 non-null   float64
dtypes: float64(1), object(7)
memory usage: 128.9+ KB


### Handling missing values in the test dataset
handle the same way as the train data

In [386]:
#  backward fill
dtn["Year"] = dtn["Year"].fillna(method="bfill")
dtn["Type"] = dtn["Type"].fillna(method="bfill")
dtn["Colour"] = dtn["Colour"].fillna(method="bfill")

In [387]:
# also remove the distance column
dtn.drop(columns="Distance", inplace=True)

In [388]:
dtn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   VehicleID  2061 non-null   object
 1   Location   2061 non-null   object
 2   Maker      2061 non-null   object
 3   Model      2061 non-null   object
 4   Year       2061 non-null   object
 5   Colour     2061 non-null   object
 6   Type       2061 non-null   object
dtypes: object(7)
memory usage: 112.8+ KB


In [389]:
# Also drop the vehicleid column 
dtn.drop(columns="VehicleID", inplace=True)

### Convert categorical features into numerical values

In [390]:
cat_col = ["Location", "Maker", "Model", "Year", "Colour", "Type"]

In [391]:
def encoder(df):
    for col in cat_col:
        lab = LabelEncoder()#CountEncoder(normalize=True)
        lab.fit(df[col])
        df[col] = lab.transform(df[col])

In [392]:
encoder(dfn)
encoder(dtn)

In [393]:
dtn

,Location,Maker,Model,Year,Colour,Type
0,0,2,8,14,16,1
1,2,37,123,19,16,1
2,2,37,272,11,7,1
3,2,37,123,17,8,1
4,2,20,192,19,12,1
...,...,...,...,...,...,...
2056,0,37,74,26,12,1
2057,1,1,391,17,3,1
2058,1,37,123,21,2,1
2059,0,20,302,19,16,1


### Split the Data

In [394]:
# split into X and y
target = "Amount (Million Naira)"
X = dfn.drop(columns=target)
y = dfn[target]

In [395]:
# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [396]:
X_test.shape, X_train.shape, y_train.shape, y_test.shape

((1441, 6), (5764, 6), (5764,), (1441,))

# Build Model

In [400]:
model = make_pipeline(
    SimpleImputer(),
    LinearRegression()
)

In [401]:
model.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('linearregression', LinearRegression())])

In [402]:
lr_pred = model.predict(X_test)

In [403]:
lr_pred

array([  3.5017654 ,  17.38473009, -39.46815819, ...,   5.0750743 ,
        13.82893854,   5.12721432])

In [404]:
MSE = mean_squared_error(y_test, lr_pred)

In [405]:
MSE

433.0904243365802